# Python project

### 한국어 끝말잇기 게임 구현

In [1]:
import requests
import os
import sys
from bs4 import BeautifulSoup
import re
import json
import urllib.request
from gtts import gTTS
from IPython.display  import Audio

In [2]:
filename = "words_list.txt"

def make_words():
    WORDS = []
    url = "https://ko.wiktionary.org/wiki/부록:자주_쓰이는_한국어_낱말_5800"
    r = requests.get(url)
    bs = BeautifulSoup(r.text, "lxml")
    table = bs.select_one("table.prettytable")
    dds = table.select("tr > td > dl > dd")
    for dd in dds:
        txt = dd.text.replace(" ", "").strip()
        if len(txt) > 1:
            WORDS.append(txt)
    
    return list(set(WORDS))

def save_words(WORDS):
    with open(filename, "w", encoding="utf-8") as file:
        for w in WORDS:
            file.write(w + "\n")

def load_words():
    WORDS = []
    try:
        with open(filename, "r", encoding="utf-8") as file:
            for w in file:
                WORDS.append(w.strip())
    except:
        pass
    return WORDS

def check_word(word):
    client_id = "rgYPW9_vNiB_RFHXsa3D"
    client_secret = "w3GKEr2x8_"    
    encText = urllib.parse.quote(word)
    url = "https://openapi.naver.com/v1/search/encyc?query=" + encText # JSON 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
    else:
        print("Error Code:" + rescode)
    temp_word = json.loads(response_body.decode('utf-8'))['items']
    return temp_word

def gt(word, lang, toSlow=True):
    tts = gTTS(text=word, lang=lang, slow=toSlow)
    tts.save('say.wav')
    display(Audio('say.wav', autoplay=True))

In [3]:
WORDS = load_words()
if len(WORDS) == 0:
    WORDS = make_words()
    save_words(WORDS)

HISTORY_USER = []
HISTORY_COM = []

In [4]:
def main():
    global WORDS, HISTORY_COM, HISTORY_USER
    gt("진행할 게임 수를 입력하세요.", 'ko', False)
    ROUND = 0
    while True:
        ROUND = input("진행할 게임 수를 입력하세요. Round:")
        if not ROUND.isdigit():
            gt("다시 입력해 주세요.", 'ko', False)
            print('다시 입력해 주세요')
        elif int(ROUND)%2 == 0:
            gt("홀수로 입력해 주세요.", 'ko', False)
            print('홀수로 입력해 주세요')
        else :
            ROUND=int(ROUND)      
            break    

    COM_SCORE = 0
    USER_SCORE = 0
    last_word = ""
    r = 0
    while r < ROUND:
        user = input("단어를 입력하세요 (종료:q)> ")
        gt(user, lang = 'ko')
        if user == "q":
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            print("게임 종료")
            gt("게임을 종료합니다.", 'ko', False)
            break
        if len(user) <= 1:
            print("중복된 단어입니다")
            gt("두 글자이상의 단어를 입력해 주세요.", 'ko', False)
            COM_SCORE += 1
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            r += 1
            continue
        if user in HISTORY_USER:
            print("중복된 단어입니다")
            gt("중복된 단어입니다.", 'ko', False)
            COM_SCORE += 1
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            r += 1
            continue
        if last_word != "" and last_word[-1] != user[0]:
            print(f"{last_word[-1]}로 시작해야 합니다.")
            gt(f"{last_word[-1]}로 시작해야 합니다.", 'ko', False)
            COM_SCORE += 1
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            r += 1
            continue
        result = check_word(user)
        if len(result) == 0 or (len(result) > 0 and user not in result[0]['title'] and user not in result[0]['description']):
            print("존재하지 않는 단어 입니다.")
            gt("존재하지 않는 단어입니다.", 'ko', False)
            COM_SCORE += 1
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            r += 1
            continue
        HISTORY_USER.append(user)
        last_word = ""
        for W in WORDS:
            if user[-1] == W[0]:
                if W in HISTORY_COM:
                    continue
                last_word = W
                WORDS.remove(W)
                HISTORY_COM.append(W)
                gt(W, 'ko')
                print(W)
                break
        if last_word == "":
            print(">>>>> 당신이 이겼습니다. <<<<<")
            gt("   당신이 이겼습니다.", 'ko', False)
            USER_SCORE += 1
            print("User Score:", USER_SCORE)
            print("Computer Score:", COM_SCORE)
            r += 1
            continue

    if COM_SCORE < USER_SCORE:
        print("")
        print("")
        print("--------------------")
        print(">>>>> You Win! <<<<<")
        print("--------------------")
        print("User:", USER_SCORE)
        print("Computer:", COM_SCORE)
        print("------------------")
        gt("You Win!", lang = 'en')
    elif COM_SCORE > USER_SCORE:
        print("")
        print("")
        print("--------------------")
        print(">>>>> Lose! <<<<<")
        print("--------------------")
        print("User:", USER_SCORE)
        print("Computer:", COM_SCORE)
        print("------------------")
        gt("Lose!", lang = 'en')
        
main()

진행할 게임 수를 입력하세요. Round: 3
단어를 입력하세요 (종료:q)>  다음


음반


단어를 입력하세요 (종료:q)>  반장


장식


단어를 입력하세요 (종료:q)>  식기


...

>>>>> 당신이 이겼습니다. <<<<<


User: 2
Computer: 0


단어를 입력하세요 (종료:q)>  리튬


>>>>> 당신이 이겼습니다. <<<<<


User: 3
Computer: 0


------------------
>>>>> You Win! <<<<<
------------------
User: 3
Computer: 0
------------------
